In [ ]:
%matplotlib inline

import numpy as np

import menpo.io as mio
from menpo.image import Image
from menpo.feature import normalize_norm
from menpo.visualize import visualize_images

from alaborticcv2015.deepconvkernel.generative import GenerativeLDCN, learn_pca_filters

# PCA LDCN 

Choose whether to work with color or greyscale images and whether to learn the filters from landmarks or regular grid:

In [ ]:
color = False
landmarks = False

if color:
    n_channels = 3
else:
    n_channels = 1

In [ ]:
images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/trainset/', verbose=True, 
                           max_images=100):
    i.crop_to_landmarks_proportion_inplace(1)
    i = i.rescale_landmarks_to_diagonal_range(100)
    if not color:
        if i.n_channels == 3:
            i = i.as_greyscale(mode='average')
    images.append(i)

Build a PCA LinDeepConvNet:

In [ ]:
net = GenerativeLDCN(architecture=1,
                     learn_filters=learn_pca_filters, 
                     n_filters=8, 
                     n_layers=3)

if landmarks:
    net.learn_network_from_landmarks(images, group='PTS', verbose=True)
else:
    net.learn_network_from_grid(images, stride=(4, 4), verbose=True)

In [ ]:
print '# of layers:', net.n_layers
print '# of filters:', net.n_filters
print '# of filters per layer:', net.n_filters_layer
print 'filters shape:', net.filters_shape

Visualize the previous filters using the previous LinDeepConvNet:

In [ ]:
filters = net.filters_spatial()

First layer spatial filters:

In [ ]:
visualize_images(filters[0])

Second layer spatial filters:

In [ ]:
visualize_images(filters[1])

We can also visualize the previous filters on the frequency domain:

In [ ]:
filters_frequecy = net.filters_frequency(ext_shape=(50, 50))

First layer filters in the frequency domain:

In [ ]:
visualize_images(filters_frequecy[0])

Second layer filters in the frequency domain:

In [ ]:
visualize_images(filters_frequecy[1])

Visualize Deep Convolutional Kernels:

In [ ]:
kernels = net.kernels_spatial(ext_shape=(50, 50))

In [ ]:
visualize_images(kernels)

In [ ]:
kernels = net.kernels_frequency(ext_shape=(50, 50))

In [ ]:
visualize_images(kernels)

Load images:

In [ ]:
images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/testset/', verbose=True, 
                           max_images=10):
    i.crop_to_landmarks_proportion_inplace(1)
    i = i.rescale_landmarks_to_diagonal_range(100)
    if not color:
        if i.n_channels == 3:
            i = i.as_greyscale(mode='average')
    images.append(i)

Choose one of the previous images:

In [ ]:
image = images[0]

Compute network features:

In [ ]:
net.network_response(image).view_widget()

Compute deep convolutional kernel features:

In [ ]:
net.kernel_response(image).view_widget()

Check Network and Kernel equivalence:

In [ ]:
network_r = net.network_response(image)
kernel_r = net.kernel_response(image)

print network_r.as_vector().dot(network_r.as_vector())
print kernel_r.as_vector().dot(kernel_r.as_vector())